# Using Mixtures

The notebook explains how to use the Mixture classes to create *Phase* objects with the thermophysical properties arising from the properties and compositions of the individual components.

In [1]:
import openpnm as op
pn = op.network.Demo()

## Create individual species objects

Let's illustrate this
by creating ``air`` as a mixture of oxygen and nitrogen.  First we create a ``Phase`` object for each individual species.  OpenPNM uses a package called [chemicals](https://pypi.org/project/chemicals/) to provide physical properties for a large variety of pure species. So we can create a ``Species`` object "by name" which will fetch the corresponding properties from a database:

In [2]:
o2 = op.phase.GasByName(network=pn, species='oxygen')
n2 = op.phase.GasByName(network=pn, species='nitrogen')

AttributeError: module 'openpnm.phase' has no attribute 'GasByName'

The ``species`` argument accepts a variety of names for a given species, so that ``species='o2'`` would also work.  We can print these objects to see what properties were calculated:

In [3]:
print(o2)

NameError: name 'o2' is not defined

This computes a few basic properties, but more importantly, the ``params`` attribute is loaded with a physical constants:

In [4]:
print(o2.params)

NameError: name 'o2' is not defined

These constants are used for computation of more complex properties, especially for mixtures.

**Pro Tip**: You can access the values of these parameters either from the ``params`` attribute or using the dictionary lookup of the main object which will dispatch the query to the ``params`` attribute.  Writing also works.

In [5]:
o2.params['CAS']

NameError: name 'o2' is not defined

In [6]:
o2['param.CAS']

NameError: name 'o2' is not defined

In [7]:
o2['param.foo'] = 'bar'
o2.params['foo']

NameError: name 'o2' is not defined

## Create a mixture object

The ``phase`` module contains a few classes designed to work with mixtures.  These basically keep track of which individual species are in the mixture and their compositions, and they have a suite of pore-scale models that compute the mixture properties using common mixing rules or correlations:

In [8]:
air = op.phase.BinaryGas(network=pn, components=[o2, n2])

NameError: name 'o2' is not defined

The ``air`` object is not quite ready yet because we have not specified the composition of each species.  This can be done a few  ways, but a useful and powerful method is available, ``y``, which allows both *setting* the mole fraction (conventionally referred to as *y* in chemical engineering) and *getting* the current values. When a component name *and* a mole fraction is given it sets this value:

In [9]:
air.y(o2.name, 0.21)
air.y(n2.name, 0.79)

NameError: name 'air' is not defined

The values of mole fraction are stored in the ``Mixture`` object with the species names as part of the key:

In [10]:
print(air['pore.mole_fraction.'+o2.name])
print(air['pore.mole_fraction.'+n2.name])

NameError: name 'air' is not defined

**Pro Tip**: The model fractions of *both* species can be retrieved using the partial dict lookup:

In [11]:
print(air['pore.mole_fraction'])

NameError: name 'air' is not defined

**Pro Tip**: The above trick can also be used to get a list of components that are currently part of the mixture:

In [12]:
air['pore.mole_fraction'].keys()

NameError: name 'air' is not defined

**Pro Tip**: The individual species can be retrieved using the ``components`` attribute:

In [13]:
air.components

NameError: name 'air' is not defined

The ``y`` method (and ``x`` on ``LiquidMixture``) can also be used to retrieve the mole fraction of one or both components:

In [14]:
air.y()

NameError: name 'air' is not defined

In [15]:
air.y(n2.name)

NameError: name 'air' is not defined

In addition to keeping track of the species and their compositions, the mixture object also computes mixture properties. These can be seen by printing ``air``:

In [16]:
print(air)

NameError: name 'air' is not defined

Several of the properties listed above were on the species objects, such as viscosity.  These viscosity of the mixture is computing using a mole-weigthed approach.  The molecular weight is also the mole-weighted average of the two species.  Diffusivity on the other hand was not present on the species.  It is computed for the mixture using the Lennard-Jones approach (as indicated the LJ items in the list). The binary diffusivity computing for air (i.e oxygen in nitrogen) is:

In [17]:
air['pore.diffusivity']

NameError: name 'air' is not defined

which is only slightly higher than the commonly accepted value of 2.09e-5.

Note that this class is specifically called ``BinaryGas`` because the LJ models are limited to binary systems.  All the other models are able to handle an arbitrary number of components, but computing diffusivity in ternary+ mixtures becomes challenging. At the time of this writing OpenPNM does not support this.

## Other features of mixtures

The various mixture classes offer a few other features, which will be explored below.

You can remove a species:

In [18]:
air.remove_comp(o2.name)
air.components

NameError: name 'air' is not defined

**Pro Tip**: A species is considered a component of a mixture *if and only if* ``'pore.mole_fraction.<species.name>'`` appears in the mixture dictionary. Adding and removing the corresponding array from the dictionary is literally how the components are defined. For instance:

In [19]:
del air['pore.mole_fraction.' + n2.name]
air.components

NameError: name 'air' is not defined

They can be readded in the same way:

In [20]:
air['pore.mole_fraction.' + n2.name] = 0.79
air.components

NameError: name 'air' is not defined

But there is a specific method for this:

In [21]:
air.add_comp(o2, mole_fraction=0.21)
air.components

NameError: name 'air' is not defined

You can also check the health of the mixture, such as whether all the mole fractions add to 1.0 each each pore:

In [22]:
print(air.check_mixture_health())

NameError: name 'air' is not defined

In [23]:
air.y(o2.name, 0.1)
print(air.check_mixture_health())

NameError: name 'air' is not defined

Lastly, the properties of the individual species can be retrieved from the mixture as follows:

In [24]:
air.get_comp_vals('pore.viscosity')

NameError: name 'air' is not defined

**Pro Tip**: It is also possible to use the ``*`` notation as follows:

In [25]:
air['pore.viscosity.*']

NameError: name 'air' is not defined

The reason the ``*`` notation is needed is that ``'pore.viscosity'`` is already taken by the mixture viscosity.  Ideally, ``air['pore.viscosity']`` should trigger the lookup of the species values. It was briefly considered that all mixture properties should be labelled ``'pore.viscosity.mixture'``, which would have allowed for ``air['pore.viscosity']`` to trigger the return of the mixture viscosity and all components in a dictionary, but this added a lot of complications behind the scenes so the idea was dropped. As a work around, the ``*`` notation was added to the ``__getitem__`` method such that any dictionary lookups with a key that ends in ``*`` trigger a lookup of the requested property on any component species.

## Summary

The mixture classes and functionality has been in *beta* for quite some time, and the version appearing in V3 is very streamlined and simple. 